<a href="https://colab.research.google.com/github/Leostark95/Proyecto_Competencia_Kaggle/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='056938'> Importar de librerías
---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy
import math
import os
import zipfile
import joblib
import warnings

warnings.filterwarnings("ignore")

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

In [2]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'

In [3]:
!kaggle competitions download -c udea-ai4eng-20242

 40% 8.00M/20.1M [00:00<00:00, 83.3MB/s]
100% 20.1M/20.1M [00:00<00:00, 126MB/s] 


In [4]:
!unzip udea-ai4eng-20242

Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [5]:
warnings.simplefilter(action='ignore', category=FutureWarning)

# <font color='056938'> Cargar base datos
---


- ID: Identificador único de cada estudiante.
- PERIODO: Año y periodo de la prueba (numérica).
- ESTU_PRGM_ACADEMICO: Programa académico del estudiante (categórica).
- ESTU_PRGM_DEPARTAMENTO: Departamento donde estudia el estudiante (categórica).
- ESTU_VALORMATRICULAUNIVERSIDAD: Rango del valor de la matrícula universitaria (categórica, algunos valores faltantes).
- ESTU_HORASSEMANATRABAJA: Horas semanales que el estudiante trabaja (categórica, algunos valores faltantes).
- FAMI_ESTRATOVIVIENDA: Estrato socioeconómico (categórica, algunos valores faltantes).
- FAMI_TIENEINTERNET: Si el hogar del estudiante tiene internet (categórica, algunos valores faltantes).
- FAMI_EDUCACIONPADRE: Nivel educativo del padre (categórica, algunos valores faltantes).
- FAMI_EDUCACIONMADRE: Nivel educativo de la madre (categórica, algunos valores faltantes).
- ESTU_PAGOMATRICULAPROPIO: Si el estudiante paga su matrícula con sus propios recursos (categórica, algunos valores faltantes).
- RENDIMIENTO_GLOBAL: Variable objetivo, que indica el rendimiento del estudiante (bajo, medio-bajo, medio-alto, alto)

In [6]:
# leer base de datos

df = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')

# <font color='056938'> Descripción de datos
---

In [7]:
# Tamaño del dataframe

print(df.shape)
print(df_test.shape)

(692500, 12)
(296786, 12)


In [8]:
# Información del dataframe

print(df.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 12 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ID                              692500 non-null  int64 
 1   PERIODO                         692500 non-null  int64 
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object
 7   FAMI_TIENEINTERNET              665871 non-null  object
 8   FAMI_EDUCACIONPADRE             669322 non-null  object
 9   FAMI_EDUCACIONMADRE             668836 non-null  object
 10  ESTU_PAGOMATRICULAPROPIO        686002 non-null  object
 11  RENDIMIENTO_GLOBAL              692500 non-null  object
dtypes: int64(2), object(10)
memory

In [9]:
# Estadísticas descriptivas de variables numéricas

df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,692500.0,494606.130576,285585.209455,1.0,247324.75,494564.5,741782.5,989286.0
PERIODO,692500.0,20198.366679,10.535037,20183.0,20195.00,20195.0,20203.0,20213.0


In [10]:
# Estadísticas descriptivas de variables categóricas

df.describe(include=['object']).T

,count,unique,top,freq
ESTU_PRGM_ACADEMICO,692500,948,DERECHO,53244
ESTU_PRGM_DEPARTAMENTO,692500,31,BOGOTÁ,282159
ESTU_VALORMATRICULAUNIVERSIDAD,686213,8,Entre 1 millón y menos de 2.5 millones,204048
ESTU_HORASSEMANATRABAJA,661643,5,Más de 30 horas,249352
FAMI_ESTRATOVIVIENDA,660363,7,Estrato 2,232671
FAMI_TIENEINTERNET,665871,2,Si,592514
FAMI_EDUCACIONPADRE,669322,12,Secundaria (Bachillerato) completa,128289
FAMI_EDUCACIONMADRE,668836,12,Secundaria (Bachillerato) completa,141744
ESTU_PAGOMATRICULAPROPIO,686002,2,No,382201
RENDIMIENTO_GLOBAL,692500,4,alto,175619


# <font color='056938'> Preprocesamiento de datos
---

## <font color='157699'> Limpieza de datos

In [11]:
# Datos duplicados

df.duplicated().sum()

0

In [12]:
# Datos duplicados

df_test.duplicated().sum()

0

In [13]:
# Datos faltantes por variable

df.isnull().sum()

,0
ID,0
PERIODO,0
ESTU_PRGM_ACADEMICO,0
ESTU_PRGM_DEPARTAMENTO,0
ESTU_VALORMATRICULAUNIVERSIDAD,6287
ESTU_HORASSEMANATRABAJA,30857
FAMI_ESTRATOVIVIENDA,32137
FAMI_TIENEINTERNET,26629
FAMI_EDUCACIONPADRE,23178
FAMI_EDUCACIONMADRE,23664


In [14]:
# Datos faltantes por variable

df_test.isnull().sum()

,0
Unnamed: 0,0
ID,0
PERIODO,0
ESTU_PRGM_ACADEMICO,0
ESTU_PRGM_DEPARTAMENTO,0
ESTU_VALORMATRICULAUNIVERSIDAD,2723
ESTU_HORASSEMANATRABAJA,13379
FAMI_ESTRATOVIVIENDA,13795
FAMI_TIENEINTERNET,11539
FAMI_EDUCACIONPADRE,9993


In [15]:
# Datos faltantes por variable en porcentaje

missing_values_percentage = df.isnull().mean() * 100
missing_values_percentage.round(2)

,0
ID,0.00
PERIODO,0.00
ESTU_PRGM_ACADEMICO,0.00
ESTU_PRGM_DEPARTAMENTO,0.00
ESTU_VALORMATRICULAUNIVERSIDAD,0.91
ESTU_HORASSEMANATRABAJA,4.46
FAMI_ESTRATOVIVIENDA,4.64
FAMI_TIENEINTERNET,3.85
FAMI_EDUCACIONPADRE,3.35
FAMI_EDUCACIONMADRE,3.42


In [16]:
# Datos faltantes por variable en porcentaje

missing_values_percentage = df_test.isnull().mean() * 100
missing_values_percentage.round(2)

,0
Unnamed: 0,0.00
ID,0.00
PERIODO,0.00
ESTU_PRGM_ACADEMICO,0.00
ESTU_PRGM_DEPARTAMENTO,0.00
ESTU_VALORMATRICULAUNIVERSIDAD,0.92
ESTU_HORASSEMANATRABAJA,4.51
FAMI_ESTRATOVIVIENDA,4.65
FAMI_TIENEINTERNET,3.89
FAMI_EDUCACIONPADRE,3.37


In [17]:
# Imputación por el valor más frecuente

cat_col = df.select_dtypes(include=['object']).columns
imputer = SimpleImputer(strategy='most_frequent')
df[cat_col] = imputer.fit_transform(df[cat_col])

print(df[cat_col].isnull().sum())


ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_EDUCACIONMADRE               0
ESTU_PAGOMATRICULAPROPIO          0
RENDIMIENTO_GLOBAL                0
dtype: int64


In [18]:
# Imputación por el valor más frecuente

cat_col = df_test.select_dtypes(include=['object']).columns
imputer = SimpleImputer(strategy='most_frequent')
df_test[cat_col] = imputer.fit_transform(df_test[cat_col])

print(df_test[cat_col].isnull().sum())

ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_EDUCACIONMADRE               0
ESTU_PAGOMATRICULAPROPIO          0
dtype: int64


## <font color='157699'> Transformación de datos

In [19]:
# Convertir valores que no son contextualmente numéricos a categóricos

df[['ID', 'PERIODO']] = df[['ID', 'PERIODO']].astype('object')
df_test[['ID', 'PERIODO']] = df_test[['ID', 'PERIODO']].astype('object')

## <font color='157699'> Codificación de variables categóricas ordinales

In [20]:
# Valores de la variable "ESTU_VALORMATRICULAUNIVERSIDAD"

df['ESTU_VALORMATRICULAUNIVERSIDAD'].unique()

array(['Entre 5.5 millones y menos de 7 millones',
       'Entre 2.5 millones y menos de 4 millones',
       'Entre 4 millones y menos de 5.5 millones', 'Más de 7 millones',
       'Entre 1 millón y menos de 2.5 millones',
       'Entre 500 mil y menos de 1 millón', 'Menos de 500 mil',
       'No pagó matrícula'], dtype=object)

In [21]:
# Reemplazar valores de la variable "ESTU_VALORMATRICULAUNIVERSIDAD"

val_mat = {
    'Menos de 500 mil': 1,
    'Entre 500 mil y menos de 1 millón': 2,
    'Entre 1 millón y menos de 2.5 millones': 3,
    'Entre 2.5 millones y menos de 4 millones': 4,
    'Entre 4 millones y menos de 5.5 millones': 5,
    'Entre 5.5 millones y menos de 7 millones': 6,
    'Más de 7 millones': 7,
    'No pagó matrícula': 0
}

df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(val_mat)
df_test['ESTU_VALORMATRICULAUNIVERSIDAD'] = df_test['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(val_mat)

In [22]:
# Valores de la variable "ESTU_HORASSEMANATRABAJA"

df['ESTU_HORASSEMANATRABAJA'].unique()

array(['Menos de 10 horas', '0', 'Más de 30 horas', 'Entre 21 y 30 horas',
       'Entre 11 y 20 horas'], dtype=object)

In [23]:
# Reemplazar valores de la variable "ESTU_HORASSEMANATRABAJA"

hora_trab = {
    '0': 0,
    'Menos de 10 horas': 1,
    'Entre 11 y 20 horas': 2,
    'Entre 21 y 30 horas': 3,
    'Más de 30 horas': 4
}

df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].replace(hora_trab)
df_test['ESTU_HORASSEMANATRABAJA'] = df_test['ESTU_HORASSEMANATRABAJA'].replace(hora_trab)


In [24]:
# Valores de la variable "FAMI_ESTRATOVIVIENDA"

df['FAMI_ESTRATOVIVIENDA'].unique()

array(['Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 2', 'Estrato 1',
       'Estrato 6', 'Sin Estrato'], dtype=object)

In [25]:
# Reemplazar valores de la variable "FAMI_ESTRATOVIVIENDA"

estrato = {
    'Sin Estrato': 0,
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6,
}

df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].replace(estrato)
df_test['FAMI_ESTRATOVIVIENDA'] = df_test['FAMI_ESTRATOVIVIENDA'].replace(estrato)

In [26]:
# Valores de la variable "FAMI_EDUCACIONPADRE"

df['FAMI_EDUCACIONPADRE'].unique()

array(['Técnica o tecnológica incompleta',
       'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) completa', 'No sabe',
       'Primaria completa', 'Educación profesional completa',
       'Educación profesional incompleta', 'Primaria incompleta',
       'Postgrado', 'Secundaria (Bachillerato) incompleta', 'Ninguno',
       'No Aplica'], dtype=object)

In [27]:
# Reemplazar valores de las variables "FAMI_EDUCACIONPADRE" y "FAMI_EDUCACIONMADRE"

edu_padres = {
    'No sabe': 0,
    'Ninguno': 1,
    'Primaria incompleta': 2,
    'Primaria completa': 3,
    'Secundaria (Bachillerato) incompleta': 3,  # Agrupado con Primaria completa
    'Secundaria (Bachillerato) completa': 4,
    'Técnica o tecnológica incompleta': 4,      # Agrupado con Secundaria completa
    'Técnica o tecnológica completa': 5,
    'Educación profesional incompleta': 4,      # Agrupado con Secundaria completa
    'Educación profesional completa': 6,
    'Postgrado': 7,
    'No Aplica': 0
}

df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].replace(edu_padres)
df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].replace(edu_padres)
df_test['FAMI_EDUCACIONPADRE'] = df_test['FAMI_EDUCACIONPADRE'].replace(edu_padres)
df_test['FAMI_EDUCACIONMADRE'] = df_test['FAMI_EDUCACIONMADRE'].replace(edu_padres)

In [28]:
# Reemplazar valores de las variables binarias

var_bin = {
    'Si': 1,
    'No': 0
}

# Aplicar el diccionario de mapeo a ambas columnas
df['FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].replace(var_bin)
df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].replace(var_bin)
df_test['FAMI_TIENEINTERNET'] = df_test['FAMI_TIENEINTERNET'].replace(var_bin)
df_test['ESTU_PAGOMATRICULAPROPIO'] = df_test['ESTU_PAGOMATRICULAPROPIO'].replace(var_bin)

In [29]:
# Valores de la variable "RENDIMIENTO_GLOBAL"

df['RENDIMIENTO_GLOBAL'].unique()

array(['medio-alto', 'bajo', 'alto', 'medio-bajo'], dtype=object)

In [30]:
# Reemplazar valores de la variable "ESTU_HORASSEMANATRABAJA"

rendimiento = {
    'bajo': 0,
    'medio-bajo': 1,
    'medio-alto': 2,
    'alto': 3,
}

df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].replace(rendimiento)

In [31]:
df

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,6,1,3,1,4,7,0,2
1,645256,20212,DERECHO,ATLANTICO,4,0,3,0,5,4,0,0
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,4,4,3,1,4,4,0,0
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,5,0,4,1,0,4,0,3
4,989032,20212,PSICOLOGIA,ANTIOQUIA,4,3,3,1,3,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,2,2,2,1,4,3,1,2
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,4,4,3,1,2,3,0,0
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,3,1,3,1,4,3,1,1
692498,986620,20195,PSICOLOGIA,TOLIMA,4,1,1,0,3,3,1,0


In [32]:
df_test

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,0,550236,20183,TRABAJO SOCIAL,BOLIVAR,1,1,3,1,5,3,1
1,1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,4,3,2,1,4,5,0
2,2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,3,0,3,1,3,4,0
3,3,782980,20195,CONTADURIA PUBLICA,SUCRE,3,3,1,0,2,2,0
4,4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,4,2,2,1,4,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
296781,296781,496981,20195,ADMINISTRACION DE EMPRESAS,BOGOTÁ,4,4,1,1,2,2,1
296782,296782,209415,20183,DERECHO,META,3,0,4,1,6,6,0
296783,296783,239074,20212,DERECHO,BOGOTÁ,4,4,3,1,4,6,0
296784,296784,963852,20195,INGENIERIA AERONAUTICA,ANTIOQUIA,6,2,3,1,6,6,0


In [33]:
df_test = df_test.drop(columns=['Unnamed: 0'])

## <font color='157699'> Categorización de programas acádemicos por áreas


In [34]:
# Definir un diccionario para clasificar los programas en diferentes áreas de conocimiento

areas_conocimiento = {
    'Ingenierías': ['INGENIERIA', 'INGENIERÍA', 'INDUSTRIAL', 'CIVIL', 'MECANICA', 'MECATRONICA', 'ELECTRONICA', 'ELECTRÓNICA', 'AGROINDUSTRIAL', 'AMBIENTAL'],
    'Ciencias Sociales': ['DERECHO', 'PSICOLOGIA', 'PSICOLOGÍA', 'SOCIOLOGIA', 'CIENCIA POLITICA', 'POLITICAS',
                          'HOTELERIA', 'TURISMO', 'COMUNICACION', 'SOCIAL', 'CONTADURIA PUBLICA', 'TRABAJO SOCIAL','SOCIOLOGÍA'],
    'Administración y Economía': ['ADMINISTRACIÓN', 'ADMINISTRACION', 'ECONOMIA', 'CONTADURIA', 'CONTADURÍA',
                                  'MERCADEO', 'NEGOCIOS', 'FINANCIERA', 'ADMINISTRACION DE EMPRESAS',
                                  'ADMINISTRACIÓN EN SALUD OCUPACIONAL', 'FINANZAS'],
    'Ciencias Exactas': ['MATEMATICAS', 'MATEMÁTICAS', 'FISICA', 'FÍSICA', 'QUIMICA', 'QUÍMICA', 'ESTADISTICA', 'ESTADÍSTICA', 'GEOLOGIA', 'GEOLOGÍA'],
    'Ciencias de la Salud': ['MEDICINA', 'ENFERMERIA', 'ODONTOLOGIA', 'FISIOTERAPIA', 'BIOQUIMICA', 'FARMACIA',
                             'INSTRUMENTACION QUIRURGICA', 'ZOOTECNIA', 'VETERINARIA','TERAPIA','FARMACIA','BIOQUÍMICA','NUTRICION','OPTOMETRIA', 'ODONTOLOGÍA'],
    'Arte y Humanidades': ['ARTE', 'HISTORIA', 'FILOSOFIA', 'LITERATURA', 'MAESTRO EN MÚSICA', 'ARTES ESCÉNICAS'],
    'Educación': ['PEDAGOGIA', 'LICENCIATURA'],
    'Licenciaturas': [
        'LICENCIATURA EN LENGUA CASTELLANA,INGLES Y FRANCES',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA,RECREACION Y DEPORTES',
        'LICENCIATURA EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL',
        'LICENCIATURA EN EDUCACION CON ENFASIS EN CIENCIAS SOCIALES Y AMBIENTALES',
        'LICENCIATURA EN EDUCACION  FISICA RECREACION  Y  DEPORTE',
        'LICENCIATURA EN CIENCIAS SOCIALES',
        'LICENCIATURA EN BIOLOGIA Y EDUCACION AMBIENTAL',
        'LICENCIATURA EN FILOSOFIA Y EDUCACION RELIGIOSA',
        'LICENCIATURA EN IDIOMA EXTRANJERO - INGLÉS'],
    'Arquitectura': ['ARQUITECTURA'],
    'Seguridad': ['SEGURIDAD', 'SALUD OCUPACIONAL', 'RIESGOS LABORALES'],
    'Metalurgia': ['METALURGIA', 'MATERIALES', 'MINAS', 'MINERIA', 'MINERÍA'],
    'Ciencias Naturales': ['BIOLOGIA', 'BIOLOGÍA', 'ECOLOGIA', 'ECOLOGÍA', 'CIENCIAS NATURALES', 'AMBIENTALES'],
    'Diseño': ['DISEÑO GRAFICO', 'DISEÑO INDUSTRIAL', 'DISEÑO DE MODAS', 'DISEÑO DE INTERIORES', 'DISEÑO'],

}

# Función para clasificar cada programa en su área correspondiente

def clasificar_programa(programa):
    for area, keywords in areas_conocimiento.items():
        if any(keyword in programa.upper() for keyword in keywords):
            return area
    return 'Otras Áreas'  # Para los programas que no encajen en las categorías anteriores

df['AREA_CONOCIMIENTO'] = df['ESTU_PRGM_ACADEMICO'].apply(clasificar_programa)
df_test['AREA_CONOCIMIENTO'] = df_test['ESTU_PRGM_ACADEMICO'].apply(clasificar_programa)


cols = list(df.columns)
cols = list(df_test.columns)
cols.insert(3, cols.pop(cols.index('AREA_CONOCIMIENTO')))  # Mover la columna a la cuarta posición
data = df[cols]
data = df_test[cols]

In [35]:
#Borrar la columna de ESTU_PRGM_ACADEMICO

df = df.drop(columns=['ESTU_PRGM_ACADEMICO'])
df_test = df_test.drop(columns=['ESTU_PRGM_ACADEMICO'])

## <font color='157699'> Codificación de variables categóricas nominales

In [36]:
# Dumizar la variavle "ESTU_PRGM_DEPARTAMENTO"

df = pd.get_dummies(df, columns=['ESTU_PRGM_DEPARTAMENTO', 'AREA_CONOCIMIENTO'], dtype=int)
df_test = pd.get_dummies(df_test, columns=['ESTU_PRGM_DEPARTAMENTO', 'AREA_CONOCIMIENTO'], dtype=int)

## <font color='157699'> Dataframe final para modelos

In [37]:
# Eliminar columnas innecesarias

df.drop(columns=['ID','PERIODO'], inplace=True)

In [38]:
#Tamaño del dataframe FINAL DESPUES
print(df.shape)
print(df_test.shape)

(692500, 51)
(296786, 52)


In [39]:
df

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL,ESTU_PRGM_DEPARTAMENTO_AMAZONAS,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,...,AREA_CONOCIMIENTO_Arte y Humanidades,AREA_CONOCIMIENTO_Ciencias Exactas,AREA_CONOCIMIENTO_Ciencias Naturales,AREA_CONOCIMIENTO_Ciencias Sociales,AREA_CONOCIMIENTO_Ciencias de la Salud,AREA_CONOCIMIENTO_Diseño,AREA_CONOCIMIENTO_Educación,AREA_CONOCIMIENTO_Ingenierías,AREA_CONOCIMIENTO_Otras Áreas,AREA_CONOCIMIENTO_Seguridad
0,6,1,3,1,4,7,0,2,0,0,...,0,0,0,0,1,0,0,0,0,0
1,4,0,3,0,5,4,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,4,4,3,1,4,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,4,1,0,4,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,3,3,1,3,3,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,2,2,2,1,4,3,1,2,0,0,...,0,0,1,0,0,0,0,0,0,0
692496,4,4,3,1,2,3,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
692497,3,1,3,1,4,3,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
692498,4,1,1,0,3,3,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [40]:
df_test

,ID,PERIODO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_DEPARTAMENTO_AMAZONAS,...,AREA_CONOCIMIENTO_Arte y Humanidades,AREA_CONOCIMIENTO_Ciencias Exactas,AREA_CONOCIMIENTO_Ciencias Naturales,AREA_CONOCIMIENTO_Ciencias Sociales,AREA_CONOCIMIENTO_Ciencias de la Salud,AREA_CONOCIMIENTO_Diseño,AREA_CONOCIMIENTO_Educación,AREA_CONOCIMIENTO_Ingenierías,AREA_CONOCIMIENTO_Otras Áreas,AREA_CONOCIMIENTO_Seguridad
0,550236,20183,1,1,3,1,5,3,1,0,...,0,0,0,1,0,0,0,0,0,0
1,98545,20203,4,3,2,1,4,5,0,0,...,0,0,0,0,0,0,0,0,0,0
2,499179,20212,3,0,3,1,3,4,0,0,...,0,0,0,0,0,0,0,1,0,0
3,782980,20195,3,3,1,0,2,2,0,0,...,0,0,0,1,0,0,0,0,0,0
4,785185,20212,4,2,2,1,4,4,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,496981,20195,4,4,1,1,2,2,1,0,...,0,0,0,0,0,0,0,0,0,0
296782,209415,20183,3,0,4,1,6,6,0,0,...,0,0,0,1,0,0,0,0,0,0
296783,239074,20212,4,4,3,1,4,6,0,0,...,0,0,0,1,0,0,0,0,0,0
296784,963852,20195,6,2,3,1,6,6,0,0,...,0,0,0,0,0,0,0,1,0,0


In [41]:
df.to_csv('preprocessed_train.csv', index=False)
df_test.to_csv('preprocessed_test.csv', index = False)